# RBX
## Övergripande arbetsgång
    * Importerar punkter från DGN-fil i arbetsmapp till den aktuella databasen.
      Hämtar in punkter till databasen och placerar dem under /RBX_clean
    * Beräknar avstånd från importerade punkter till fasledningar.
      Fasledningar har tidigare hämtats in från Wires.ipynb
    * I RBX_polygon.py skapas polygoner, grupperar punkter och hämtar de punkter som har minst avstånd till fas.
      Polygoner och punkter skrivs ut till shapefiler (.shp)
    * Polygoner och punkter hämtas in till geodatabasen, från shapefilerna, och lägger även till domäner, ledningsgata(nr)
      och littera.

## Sökvägar

In [6]:
# Läs settings
import json
import os
import arcpy

settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEKNNO.json"

with open(settings_file, 'r', encoding='utf-8') as file:
    settings = json.load(file)

print(settings)

run_ID = settings["run_ID"]
powerline_list = settings["powerline_list"]
local_dir = settings["local_folder"]
powerlines_folder = settings["powerlines_folder"]
domains_folder = settings["domains_folder"]
dgn_dir = settings["dgn_folder"]
scandate_file = settings["scandate_file"]
working_gdb_template = settings["working_gdb_template"]
results_gdb_template = settings["results_gdb_template"]
cvd_LEDNINGSGATA_path = os.path.join(domains_folder, "cvd_LEDNINGSGATA.txt")
wires_gdb = os.path.join(local_dir, f"wires_{run_ID}.gdb")
RBX_shape_folder = os.path.join(local_dir, f"RBX_shp_{run_ID}")
working_gdb = os.path.join(local_dir, f"working_{run_ID}.gdb")
results_gdb = os.path.join(local_dir, f"results_{run_ID}.gdb")

sr = arcpy.SpatialReference("SWEREF99_TM", "RH2000")

{'run_ID': '250128', 'powerline_list': 'Q:/Projekt/Data_2024/styrfiler/veg_kanttrad_250128.txt', 'scandate_file': 'Q:/Projekt/Data_2024/skanningsdatum/skanningsdatum_250128.txt', 'local_folder': 'C:/SVK_2024/pythonkörningar', 'working_gdb_template': 'Q:/Projekt/Data_2024/working_template.gdb', 'results_gdb_template': 'Q:/Projekt/Data_2024/results_template.gdb', 'wires_gdb_template': 'Q:/Projekt/Data_2024/wires_template.gdb', 'dgn_folder': 'Q:/Projekt/Analys_2024/DGN_2024', 'powerlines_folder': 'Q:/Projekt/Analys_2024/ledningar', 'domains_folder': 'Q:/Projekt/Data_2024/Underlag_SVK/Domains', 'LG_polygons': 'Q:/Projekt/Data_2024/Underlag_SVK/Bestallningsunderlag_2024.gdb/GNG_LEDNINGSGATA', 'station_polygons': 'Q:/Projekt/Data_2024/Underlag_SVK/Bestallningsunderlag_2024.gdb/GNG_STATIONSOMRADE'}


In [8]:
from pathlib import Path
import os
import shutil

# Create empty folder for shapefiles
if os.path.exists(RBX_shape_folder):
    print(f"{RBX_shape_folder} finns redan")
else:
    os.makedirs(RBX_shape_folder)

# Create working gdb as copy of template
if os.path.exists(working_gdb):
    print(f"{working_gdb} finns redan")
else: 
    shutil.copytree(working_gdb_template, working_gdb)

# Create results gdb as copy of template
if os.path.exists(results_gdb):
    print(f"{results_gdb} finns redan")
else: 
    shutil.copytree(results_gdb_template, results_gdb)

    
## Flytta sökvägar till settings-fil
#prj_dir = Path(r"C:\SVK_2024\pythonkörningar")
#LGs_info_path = Path(r"Q:\Projekt\Data_2024\styrfiler\veg_kanttrad_241217.txt")
#gdb_name = 'veg_kantträd_241217.gdb'
#gdb = os.path.join(prj_dir, gdb_name)
#wires_gdb = r"C:\SVK_2024\pythonkörningar\Wires_241217.gdb"
#RBX_polygon_dir = os.path.join(prj_dir, "RBX_polygons")
#serv_prj_dir = Path(r"Q:\Projekt\Analys_2024\ledningar")
#dgn_dir = Path(r"Q:\Projekt\Analys_2024\DGN_2024")
#cvd_LEDNINGSGATA_path = r"C:\Users\SEKNNO\OneDrive - Sweco AB\SVK_2024\30047770-SVK-Analys-av-laserdata-extern\python\gdb-mallar-och-domänkoder\cvd_LEDNINGSGATA.txt"
#cvd_DATE_path = r"Q:\Projekt\Data_2024\skanningsdatum\skanningsdatum_241217_temp.txt"


C:/SVK_2024/pythonkörningar\RBX_shp_250128 finns redan
C:/SVK_2024/pythonkörningar\working_250128.gdb finns redan
C:/SVK_2024/pythonkörningar\results_250128.gdb finns redan


# Slår ihop RBX-filer för blocken till en enda fil för ledningen
### Behöver inte köras, den som gör ledningen i TerraScan gör detta själv

In [2]:
#import os
#import sys
#import glob
#import fileinput
#import SVK
#import pandas as pd

In [ ]:
## Merge RBX txt files for all blocks of a powerline (LGXXX_Y) into one txt
#import fileinput
#
#def merge_blocks(src_dir, search_pattern, dst_file):
#    blocks = glob.glob(os.path.join(src_dir, search_pattern))
#    with open(dst_file, "w") as fh:
#        input_lines = fileinput.input(blocks)
#        fh.writelines(input_lines)
#        
#def combine_blocks(row):
#    LG = row["LG"]
#    line = row["line"]
#    line_dir = serv_prj_dir / LG / f"line_{line}"
#    
#    print(f"Doing {LG}_{line}")
#    block_dir = os.path.join(line_dir, "RBX", "block")
#    combined_blocks_path = os.path.join(line_dir, "RBX", "RBX_raw.txt")
#    merge_blocks(block_dir, "*_pt*.txt", combined_blocks_path)
#
#LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
#print(LGs_info)
#LGs_info.apply(combine_blocks, axis=1)

# Denna kopierar över alla rbx_raw.txt till den temporära mappen på skrivbordet
### Är det nödvändigt? Jag tror att det som gör skillnad för hastigheten är om gdb:er / shapefiler ligger lokalt istället för på server. /Karin

In [3]:
#import os
#import filecmp
#import shutil
#import pandas as pd
#df = pd.DataFrame()
#to_dir = os.path.join(prj_dir, "ledningar")
#from_dir = serv_prj_dir
#
##LGs_info_path = os.path.join(prj_dir, LGs_info_name)
#
#LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
#testlist = []
#for i, j in zip(LGs_info["LG"], LGs_info["line"]):
#    testlist.append(f'{i}\\line_{j}\\RBX')
#
## print(testlist)
#outlist_to_dir = []
#for i in testlist:
#    outlist_to_dir.append((f"{to_dir}\{i}\RBX_raw.txt").replace('\\\\', '\\'))
#
#outlist_from_dir = []
#for i in testlist:
#    outlist_from_dir.append(
#        (f"{from_dir}\{i}\RBX_raw.txt").replace('\\\\', '\\'))
#
#for i, j in zip(outlist_from_dir, outlist_to_dir):
#    if os.path.exists(j.replace('\\\\', '\\')):
#        shutil.copyfile(i.replace('\\\\', '\\'), j.replace('\\\\', '\\'))
#    else:
#        folder1 = j.split("\\")[-4]
#        folder2 = j.split("\\")[-3]
#        folder3 = j.split("\\")[-2]
#        os.makedirs(
#            os.path.join(
#                to_dir,
#                folder1,
#                folder2,
#                folder3
#            )
#        )
#        shutil.copyfile(i.replace('\\\\', '\\'), j.replace('\\\\', '\\'))
#
#for i, j in zip(outlist_from_dir, outlist_to_dir):
#    print(filecmp.cmp(i, j))
#

True
True


# Importera rensade RBX från dgn till gdb
Hämtar in punkter från DGN-filer till geodatabasen. <br>
DGN-filerna ska finnas som undermapp i projektmappen. <br>
Punkter som hämtas ska ligga i lager 20 i DGN-filerna.<br>
Om punkter ligger i annat lager kommer inga punkter att importeras. <br>
Ledningsgator och ledningar som behandlas utfrån från textfil där alla ledningar som ska köras finns med. <br>
Textfilen importeras och görs till en dataframe. <br>
I geodatabasen kommer punkterna placeras under clean_RBX_points.

In [9]:
# Copy RBX from dgn to working_gdb feature class, one fc for each LGXXX_Y
import arcpy, os, glob
from pathlib import Path
import pandas as pd

output_fd = os.path.join(working_gdb, "clean_RBX_points")

def dgn_to_gdb(powerline): 
    LG = powerline["LG"]
    line = powerline["line"]
    littera = powerline["Littera"]
    
    os.chdir(dgn_dir)
    
    print(f"påbörjar {LG}_{line}")
    arcpy.env.overwriteOutput = True
    dgn = os.path.join(dgn_dir, LG+"_"+str(line)+".dgn") #
    print(dgn)

    output_fc = os.path.join(output_fd, LG+"_"+str(line)+"_RBX_clean") #var punkterna ska exporteras till

    #print(os.path.join(output_fd, LG+"_"+str(line)+"_RBX_clean"))
    # Create a value table that will hold the input feature classes for Merge
    vTab = arcpy.ValueTable()
    #print("vTab created")
    layername_RBX = dgn + '_Layer_LVL20'
    arcpy.MakeFeatureLayer_management(dgn + '/Point', layername_RBX, "\"Level\" = 20")
    #print("MakeFeatureLayer complete")
    vTab.addRow([layername_RBX, 2])
    #print("Row added to vTab")

    # Merge the CAD features into one feature class
    arcpy.env.workspace = output_fd
    arcpy.Merge_management(vTab, output_fc)
    print("vTab merged to output_fc")
    
    # ta bort fält som inte ska användas
    keep_fields = ["Shape", "OBJECTID"]
    all_fields = arcpy.ListFields(output_fc)
    fields_to_delete = [field.name for field in all_fields if field.name not in keep_fields]
    if fields_to_delete:
        arcpy.DeleteField_management(output_fc, fields_to_delete)

    arcpy.Delete_management(layername_RBX)
    print("Layer deleted")

    #arcpy.env.workspace = dgn_dir
 
powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(dgn_to_gdb, axis=1)

påbörjar LG022_1
Q:/Projekt/Analys_2024/DGN_2024\LG022_1.dgn
vTab merged to output_fc
Layer deleted
påbörjar LG022_2
Q:/Projekt/Analys_2024/DGN_2024\LG022_2.dgn
vTab merged to output_fc
Layer deleted


0    None
1    None
dtype: object

# Near 3D - avstånd till fas för RBX
Beräknar avstånd från importerade punkter till aktuell ledning med hjälp av "Near3D_3d". <br>
Lägger till avstånd till punkterna i nya fält "AVSTAND_FAS" och "AVSTAND_HORISONTELLT" för respektive punkt. <br>
Skriptet utgår från tidigare nämnda textfil för vilka ledningsgator som ska behandlas.

In [10]:
import arcpy, os, glob
arcpy.CheckOutExtension('3D')
from pathlib import Path
import pandas as pd

fd_RBX = os.path.join(working_gdb, 'clean_RBX_points')
arcpy.env.overwriteOutput = True

def dist2wire(fd_RBX, fc_name_RBX, fc_wires, radius):
    # Compute distance from RBX point to wire
    fc_RBX = os.path.join(fd_RBX, fc_name_RBX)
    arcpy.Near3D_3d(in_features=fc_RBX, near_features=fc_wires, search_radius=radius, location="NO_LOCATION", angle="NO_ANGLE", delta="NO_DELTA")
    print('done with Near3D '+fc_name_RBX)
    arcpy.DeleteField_management(in_table=fc_RBX, drop_field="NEAR_FID")
    arcpy.management.AlterField(in_table=fc_RBX, field="NEAR_DIST3", new_field_name="AVST_F")
    arcpy.management.AlterField(in_table=fc_RBX, field="NEAR_DIST", new_field_name="AVST_H")
    print('done with delete and alter field '+fc_name_RBX)
    
    
def RBX_3D_distance(powerline):
    LG = powerline["LG"]
    line = powerline["line"]
    littera = powerline["Littera"]
    
    print(f"{LG}_{line}, littera {littera}")
    
    fc_name_RBX = f"{LG}_{line}_RBX_clean"
    fc_name_wires = f"{LG}_{line}_fas"
    fc_wires = os.path.join(wires_gdb, fc_name_wires)
    
    if arcpy.Exists(os.path.join(fd_RBX, fc_name_RBX)):
        dist2wire(fd_RBX, fc_name_RBX, fc_wires, 6)
        print(f"Distance to wire computed for RBX {LG}_{line}")
    else:
        print(f"{os.path.join(fd_RBX, fc_name_RBX)} does not exist")

# The file pointed at by LGs_info_path contains only the powerlines that should be
# processed. So this file has to be updated each time we want to process a new set of
# powerlines. This is unpractical - consider a user prompt where the user can select
# which powerlines to process

powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(RBX_3D_distance, axis=1)

LG022_1, littera UL7 S5-6
done with Near3D LG022_1_RBX_clean
done with delete and alter field LG022_1_RBX_clean
Distance to wire computed for RBX LG022_1
LG022_2, littera UL7 S2-4
done with Near3D LG022_2_RBX_clean
done with delete and alter field LG022_2_RBX_clean
Distance to wire computed for RBX LG022_2


0    None
1    None
dtype: object

# Skapa fält för littera mm

In [ ]:
#def add_fields(powerline):
#    LG = powerline["LG"]
#    line = powerline["line"]
#    littera = powerline["Littera"]
#    
#    print(f"{LG}_{line}, littera {littera}")
#    
#    fc_name_RBX = f"{LG}_{line}_RBX_clean"
#
#    if arcpy.Exists(os.path.join(fd_RBX, fc_name_RBX)):
#        
#        print(f"Fields added to RBX {LG}_{line}")
#    else:
#        print(f"{os.path.join(fd_RBX, fc_name_RBX)} does not exist")
#    
#
#powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
#powerlines_df.apply(add_fields axis=1)

#  gör shp med polygoner, alla punkter, närmaste punkter
delar in punkter och polygoner i farlighetsklass
sätter minsta avstånd till fas för varje polygon. <br>

In [11]:
8# -*- coding: utf-8 -*-
"""
Created on Mon Aug 16 16:16:11 2021

@author: SEKNNO
"""
# %% imports
# imports
import os
import pandas as pd
import geopandas as gpd
import fiona
import copy
from fiona import supported_drivers
from geopandas.tools import sjoin
from geopandas.tools import overlay
from shapely.geometry import Point
from shapely.ops import unary_union
import numpy


# %% FUNCTIONS
# FUNCTIONS
def create_dir(dir_path):
    """Creating directory

    Given the path, the functions creates a directory
    if there is none.
    """
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(dir_path + " created.")
    else:
        print(dir_path + " already exists. Not overwritten.")


def buffer_points(points):
    """Buffering points.

    For the given points,
    the function creates a specified sized buffer for these.
    """
    buffer_geom = points.geometry.buffer(0.5, cap_style=1)
    buffered_points = gpd.GeoDataFrame(geometry=buffer_geom)
    return (buffered_points)


def cluster_buffers(buffers):
    """Clustering buffer objects.

    Given the buffered points, the function creates clusters
    which are used to group points and later create polygons.
    """
    # Creating GeoSeries with unions of buffered points.
    buffers_union = gpd.GeoSeries(unary_union(buffers.geometry))
    # Exploding the unions in a new GeoDataFrame.
    clustered_buffers = buffers_union.explode(index_parts=False)
    clustered_buffers = gpd.GeoDataFrame(geometry=clustered_buffers)
    # Resetting indices
    clustered_buffers.reset_index(drop=True, inplace=True)
    # Sorting out geometry columns
    #clustered_buffers.geometry = clustered_buffers.iloc[:, 2]
    #clustered_buffers = clustered_buffers[["geometry"]]
    print(f"Clustered_buffers: {clustered_buffers.head(3)}")
    # Setting coordinate system for GeoDataFrame
    clustered_buffers.set_crs("epsg:3006", inplace=True)
    # Returning cluster GDF.
    return (clustered_buffers)


def read_points(gdb, LG, line):
    """Reading information from file.

    Import points from gdb (which have had false alarms removed
    manually in MicroStation and then exported to gdb
    and Near3D (avst fas) computed)
    """
    # Defining layer name based on line number.
    point_layer = f"{LG}_{line}_RBX_clean"
    # Checking if layer name is relevant.
    if point_layer not in RBX_gdb_layers:
        # no points from which to make clusters: return nothing.
        return ([], [])
    # Reading file.
    points = gpd.read_file(gdb, layer=point_layer)
    # Creating new list with relevant coulmn.
    #points = points[["AVSTAND_FAS", "AVSTAND_HORISONTELLT", "geometry"]] ###### ÄNDRAT 2025-01-14 ######
    points = points[["AVST_F", "AVST_H", "geometry"]] ###### ÄNDRAT 2025-01-14 ######
    # Checking if lists are containing any columns.
    if len(points) == 0:
        # If empty, return empty list.
        return ([], [])
    # Returning points list.
    return (points)


def read_raw_RBX(raw_RBX_file):
    """Reading infomation from file

    Used to get the dz, the height, for points,
    which is not contained in the dgn-file.
    Reading the RAW RBX file and then adding the calculated value to 
    the actual point.
    """
    # Reading file
    raw_RBX = pd.read_csv(raw_RBX_file, header=None,
                          sep=" ", names=["x", "y", "z", "dz"])
    # Creating GeoDataFrame from the imported list and calculating
    # the dz value.
    raw_RBX_points = gpd.GeoDataFrame(
        raw_RBX, geometry=gpd.points_from_xy(
            raw_RBX.x, raw_RBX.y, (raw_RBX.z - raw_RBX.dz)))
    # Dropping the irrelevant information.
    raw_RBX_points.drop(["x", "y"], axis=1, inplace=True)
    # Returning the extended GDF with the dz value.
    return (raw_RBX_points)


def make_RBX_polygons(points, voltage, LG, littera, RBX_class):
    """Create polygons from the RBX points.

    Get max 3D distance from wire for RBX_class
    """
    RBX_dist = RBX_distances[voltage][RBX_class]
    # extract points within RBX_dist
    # RBX_class_points = points[points["AVSTAND_FAS"] <= RBX_dist] #### ÄNDRAT 2025-01-14
    RBX_class_points = points[points["AVST_F"] <= RBX_dist]
    if len(RBX_class_points) == 0:
        return ([], [])
    # buffer points, create clusters
    buffers = buffer_points(RBX_class_points)
    clusters = cluster_buffers(buffers)
    clusters['LG'] = LG
    clusters['littera'] = littera

    return (clusters, RBX_class_points)


def set_class(points, voltage, LG, littera, RBX_class, RBX_class_nr):
    """Set "KLASS_TEMP" value for points.

    Setting value based on distance to line, type of voltage and RBX class.
    """
    # Deciding which distance is to be calculated
    RBX_dist = RBX_distances[voltage][RBX_class]
    # Filtering out points whos distance is less than the acceptable.
    # RBX_class_points = points[points["AVSTAND_FAS"] <= RBX_dist] #### ÄNDRAT 2025-01-14
    RBX_class_points = points[points["AVST_F"] <= RBX_dist]
    # Filtering out points whos distance is not less than the acceptable.
    # Not edited.
    # RBX_no_class_points = points[points["AVSTAND_FAS"] > RBX_dist] #### ÄNDRAT 2025-01-14
    RBX_no_class_points = points[points["AVST_F"] > RBX_dist]
    # Set the value of class for the relevant points.
    RBX_class_points["KLASS_TEMP"] = RBX_class_nr

    # Returning the the full points list, where some points has been changed.
    return (RBX_class_points._append(RBX_no_class_points))


def clip_polygons(clippee, clipper):
    """Clip overlay polygons.

    Using the higher class polygon as template,
    the function makes a cut in the lower class polygon
    so that no overlap exist.
    """
    # Check if the there is a lower class polygon.
    # If not, the returning empty list.
    if len(clippee) == 0:
        return ([], [])
    # Check if there is a higher class polygon,
    # if not, then returning the lower class polygon unclipped.
    if len(clipper) == 0:
        return clippee
    # Clipping the the higher class polygon out of the lower class polygon.
    clipped = overlay(clippee, clipper, how="difference")
    # Returning the clipped lower class polygon
    return clipped


def calculate_RBX_area(RBX):
    """Calculating the area for polygons."""
    # Check if the there is a polygon.
    # If not, the returning empty list.
    if len(RBX) == 0:
        return 0
    # Calculating area
    area = RBX.geometry.area.sum()
    return area


def get_attributes_from_points(polygons, points):
    """

    """
    if str(type(polygons)) != "<class 'geopandas.geodataframe.GeoDataFrame'>":
        return ([], [])
    polygons["polygon_ID"] = polygons.index
    print(f"Nr of points: {len(points)}")
    print(f"Nr of polygons: {len(polygons)}")

    points_with_ID = sjoin(points, polygons, how="inner")
    # sorted_points = points_with_ID.sort_values(by='AVSTAND_FAS') #### ÄNDRAT 2025-01-14
    sorted_points = points_with_ID.sort_values(by='AVST_F')
    closest_points = sorted_points.groupby('polygon_ID').first().reset_index()
    closest_points_full = copy.deepcopy(closest_points)
    # inplace för att ändra den faktiska punkten
    closest_points.drop(["geometry", "LG", "littera"], axis=1, inplace=True)
    print(closest_points[0:5])
    print(f"Number of closest points: {len(closest_points)}")

    # kopplar på information till polygon från closest point
    # tillfällig
    polys_ = polygons.merge(closest_points, on="polygon_ID")
    print(f"Original polygon columns: {polygons.columns}")
    print(f"Merged polygon columns: {polys_.columns}")

    # kolla koppling mellan denna och poly_
    #polys = polys_[["LG", "littera", "AVSTAND_FAS",
    #                "AVSTAND_HORISONTELLT", "geometry", "dz"]] #### ÄNDRAT 2025-01-14
    polys = polys_[["LG", "littera", "AVST_F",
                    "AVST_H", "geometry", "dz"]]
    print(f"Columns: {polys.columns}")

    return (polys, closest_points_full)


def set_z_to_points(points, raw_rbx_file):
    """Setting ground level z to clean points

    Z value is changed from the original \\
    z value (z + dz value of the clean point) \\
    to the z value from the same raw_rbx point (z - dz, ground level)

    Keyword arguments: \\
    points -- dataframe of verified rbx points \\
    raw_rbx -- dataframe of all rbx points \\
    raw_rbx_file -- text document containing information about the raw points

    out : dataframe(
        AVSTAND_FAS, \\
        AVSTAND_HORISONTELLT, \\
        geometry(x, y, z(ground level))
    )
    """
    # Checking if the input data is as GeoDataFrame
    # If not, returning empty list
    if str(type(points)) != "<class 'geopandas.geodataframe.GeoDataFrame'>":
        return ([], [])
    raw_rbx = pd.read_csv(raw_rbx_file, header=None,
                          sep=" ", names=["x", "y", "z",
                                          "dz", "LG", "littera"]
                          )

    # adding columns to points dataframe for x, y values.
    points["x"] = points.centroid.map(lambda p: p.x)
    points["y"] = points.centroid.map(lambda p: p.y)

    # creating container for numeric x, y, z values.

    # 2023-02-06
    points["dz"] = numpy.float64

    pointlst = []
    point_lst_dz = []
    no_ref_points = []
    # for each point in sorted db, getting the index for further comparison
    for ind1 in range(0, len(points)):
        checklst = []
        # for each point in the raw_rbx db,
        # getting the index for further comparison
        for ind2 in range(0, len(raw_rbx)):
            # validating if the x values and the y values are the same
            # for the clean_point and raw_point resp.
            if (round(points.x[ind1], 2) == raw_rbx.x[ind2]
                    and round(points.y[ind1], 2) == raw_rbx.y[ind2]):
                # if statement is true,
                # then the points are the same in x, y direction,
                # which means the z value can be changed
                # from dz (sorted point)
                # to z (raw_rbx_point, ground level).
                pointlst.append(
                    [
                        "", "", points.x[ind1], points.y[ind1],
                        round((raw_rbx.z[ind2] - raw_rbx.dz[ind2]), 1),
                        raw_rbx.dz[ind2]
                    ]
                )
                point_lst_dz.append(raw_rbx.dz[ind2])
                checklst.append(raw_rbx.dz[ind2])
                break
            else:
                continue
        # 2023-02-06
        if not checklst:
            no_ref_points.append(points.iloc[ind1])
        # 2022
        # else:
        #    continue
    # creating a dataframe, setting headers.
    #pointlst_df = pd.DataFrame(
    #    pointlst, columns=[
    #        "AVSTAND_FAS", "AVSTAND_HORISONTELLT",
    #        "x", "y", "z", "dz"]) ###### ÄNDRAT 2025-01-14
    pointlst_df = pd.DataFrame(
        pointlst, columns=[
            "AVSTAND_FAS", "AVST_H",
            "x", "y", "z", "dz"])    

    # creating points from the numeric values in the dataframe.
    newpointlst = gpd.GeoDataFrame(pointlst_df, geometry=gpd.points_from_xy(
        pointlst_df["x"], pointlst_df["y"], pointlst_df["z"]), crs="epsg:3006"
    )
    # checking that the length of points list (list in)
    # is equal to the length of the generated list (list out)
    if len(newpointlst) != len(points):
        return None

    # adding the old information to the new dataframe.
    #newpointlst["AVSTAND_FAS"] = points["AVSTAND_FAS"]
    #newpointlst["AVSTAND_HORISONTELLT"] = points["AVSTAND_HORISONTELLT"] #### ÄNDRAT 2025-01-14
    newpointlst["AVST_F"] = points["AVST_F"]
    newpointlst["AVST_H"] = points["AVST_H"]

    
    # removing the columns for the numeric x, y, z values
    newpointlst.drop(["x", "y", "z"], axis=1, inplace=True)

    # returning the new dataframe
    return newpointlst


def round_values(gdf):
    """Rounding up values.

    When getting a GeoDataFrame,
    the funtion is rounding the value up to one decimal
    for the "AVSTAND_FAS" and "AVSTAND_HORISONTELLT"
    """
    # Sorting out the relevant columns
    #rndvals = ['AVSTAND_FAS', 'AVSTAND_HORISONTELLT', 'dz'] ##### ÄNDRAT 2025-01-14
    rndvals = ['AVST_F', 'AVST_H', 'dz']
    columns = gdf.columns
    # Rounding up the value per column
    for col in rndvals:
        if col in columns:
            gdf[col] = round(gdf[col], 1)
    # Returning updated GeoDataFrame
    return gdf


def sort_columns(gdf):
    """Sorting GeoDataFrame for consistency

    Given the output GeoDataFrame,
    the function sorts the relevant columns in right order.
    """
    # Creating list of how the GDF is going to be sorted
#    order = ['geometry', 'AVSTAND_FAS', 'AVSTAND_HORISONTELLT',
#             'dz', 'LG', "littera", 'KLASS_TEMP'] #### ÄNDRAT 2025-01-14
    order = ['geometry', 'AVST_F', 'AVST_H',
             'dz', 'LG', "littera", 'KLASS_TEMP']
    # Creating empty GDF
    out_gdf = gpd.GeoDataFrame()
    # Get all columns
    columns = gdf.columns
    # Sorting per column
    for col in order:
        if col in columns:
            out_gdf[col] = gdf[col]
    # Return sorted GDF
    return out_gdf


def RBX_polys_stats(row):
    """Main function of RBX_polygons.py"""

    # Fecthing values, row by row, from the line dataframe.
    LG = row["LG"]
    line = row["line"]
    littera = row["Littera"]
    voltage = row["Spänning"]

    # Prompting which line is handled
    print(f"Doing {LG}_{line}")

    # the cleaned RBX points that where imported from dgn to gdb
    points = read_points(working_gdb, LG, line)
    if str(type(points)) != "<class 'geopandas.geodataframe.GeoDataFrame'>":
        print("No points in list")
        return ([], [])

    # the raw RBX points from the txt file.
    # Needed because they have dZ attribute
    raw_RBX_file = os.path.join(
        powerlines_folder, LG,
        f"line_{line}", "RBX", str("RBX_raw.txt"))

    # Using "set_z_points" function to add z value to all points.
    points = set_z_to_points(points, raw_RBX_file)

    # Creating polygons from distance to line.
    if len(points) > 0:
        red_polygons, red_points = make_RBX_polygons(
            points, voltage, LG, littera, "red")
        orange_polygons, orange_points = make_RBX_polygons(
            points, voltage, LG, littera, "orange")
        yellow_polygons, yellow_points = make_RBX_polygons(
            points, voltage, LG, littera, "yellow")

    # Creating a default value and column for "KLASS_TEMP".
    points["KLASS_TEMP"] = 0
    # Using the "set_class" function for setting relevant "KLASS_TEMP" value
    # for all points.
    if len(points) > 0:
        points = set_class(
            points, voltage, LG, littera, "yellow", 1)
        points = set_class(
            points, voltage, LG, littera, "orange", 2)
        points = set_class(
            points, voltage, LG, littera, "red", 3)

    # Getting all the closest points from the unclipped yellow polygons.
    # The unclipped yellow polygons are used to cluster all points in groups,
    # sort them as descending by "AVSTAND_FAS" and get the point with smallest
    # distance for each group.
    yellow_polygons["polygon_ID"] = yellow_polygons.index
    closest_points = sjoin(yellow_points, yellow_polygons, how="inner")
    # closest_points = closest_points.sort_values(by='AVSTAND_FAS') #### ÄNDRAT 2025-01-14
    closest_points = closest_points.sort_values(by='AVST_F')
    closest_points = closest_points.groupby('polygon_ID')

    # Sorting out the closest point for each group.
    all_closest_points_lst = []
    for spt in closest_points:
        for i in spt:
            if type(i) is not int:
                all_closest_points_lst.append(i.iloc[0])

    # Cutting the polygons.
    _yellow_RBX = clip_polygons(yellow_polygons, orange_polygons)
    _orange_RBX = clip_polygons(orange_polygons, red_polygons)
    _red_RBX = red_polygons

    # Adding values to polygons from points.
    # Returning polygons and all points in groups.
    yellow_RBX_all = get_attributes_from_points(
        _yellow_RBX, yellow_points)
    orange_RBX_all = get_attributes_from_points(
        _orange_RBX, orange_points)
    red_RBX_all = get_attributes_from_points(_red_RBX, red_points)

    # Sorting relevant data from get_attributes_from_point.
    if len(yellow_RBX_all[:-1]) > 0:
        yellow_RBX = yellow_RBX_all[:-1][0]
    else:
        yellow_RBX = []
    if len(orange_RBX_all[:-1]) > 0:
        orange_RBX = orange_RBX_all[:-1][0]
    else:
        orange_RBX = []
    if len(red_RBX_all[:-1]) > 0:
        red_RBX = red_RBX_all[:-1][0]
    else:
        red_RBX = []

    RBX_all_lines["yellow"].append(yellow_RBX)
    RBX_all_lines["orange"].append(orange_RBX)
    RBX_all_lines["red"].append(red_RBX)

    # Placing "KLASS_TEMP" and "Area" values for polygons
    all_RBX_list = []
    for cnt, gdf in enumerate([yellow_RBX, orange_RBX, red_RBX]):
        if len(gdf) > 0:
            gdf['KLASS_TEMP'] = cnt+1
            gdf['area'] = gdf.geometry.area
            all_RBX_list.append(gdf)

    # for each color, calculate RBX_area and number of clusters,
    # and append to summary.
    yellow_area = calculate_RBX_area(yellow_RBX)
    orange_area = calculate_RBX_area(orange_RBX)
    red_area = calculate_RBX_area(red_RBX)

    nr_yellow = len(yellow_RBX)
    nr_orange = len(orange_RBX)
    nr_red = len(red_RBX)

    area_summary.append([LG, line, littera, yellow_area,
                        orange_area, red_area, nr_yellow, nr_orange, nr_red])

    # Creating Geodataframe out of a list
    all_RBX = gpd.GeoDataFrame(
        pd.concat(all_RBX_list, ignore_index=True), crs="epsg:3006")

    ###################################### 2024-12-18: ORDNA FÄLT HÄR NEDANFÖR? ####################################
    all_RBX["LG"] = all_RBX["LG"]
    all_RBX.drop(
        ["area", "LG", "littera"],  # , "Shape_Length", "Shape_Area"
        axis=1,
        inplace=True)

    # Rounding up values for heights and distances.
    all_RBX = round_values(all_RBX)
    all_RBX_out = sort_columns(all_RBX)
    all_RBX_out.to_file(os.path.join(
        RBX_shape_folder, f"{LG}_{line}_RBX.shp"), crs="epsg:3006")

    # Handling all the closest points.
    # Adding KLASS_TEMP values and rounding values up.
    # Creating a dataframe and writing it to a .shp-file.
    all_closest_points = gpd.GeoDataFrame(
        all_closest_points_lst, crs="epsg:3006").reset_index()
    # Creaing default value and column for "KLASS_TEMP"
    all_closest_points["KLASS_TEMP"] = int(0)
    # Using the "set_class" function for setting relevant "KLASS_TEMP" value
    # for all closest points.
    if len(all_closest_points) > 0:
        all_closest_points = set_class(
            all_closest_points, voltage, LG, littera, "yellow", 1)
        all_closest_points = set_class(
            all_closest_points, voltage, LG, littera, "orange", 2)
        all_closest_points = set_class(
            all_closest_points, voltage, LG, littera, "red", 3)
    # Cleaning up GeoDataFrame from information that is not relevant.
    all_closest_points.drop(
        ["index", "index_right", "polygon_ID", "LG", "littera"],
        axis=1,
        inplace=True)
    # Rounding up values for relevant columns.
    all_closest_points = round_values(all_closest_points)
    # Sorting columns for consistency.
    all_closest_points_out = sort_columns(all_closest_points)
    all_closest_points_out.to_file(os.path.join(
        RBX_shape_folder, f"{LG}_{line}_closest_points.shp"), crs="epsg:3006")

    # Handling all the clean RBX points.
    points = round_values(points)
    points_out = sort_columns(points)
    # points["LG"] = points["LG"].convert_dtypes('int64')
    points_out.to_file(os.path.join(
        RBX_shape_folder, f"{LG}_{line}_all_points.shp"), crs="epsg:3006")

# %% RUN
# RUN
# In RBX_distances below, yellow distance is set to 0.1 m higher than in the macro.
# arcpy NEAR_3D and TerraScan calculations can differ slightly, so that points
# with a distance to wire < 4.0 m according to TerraScan are further than 4.0 m
# from wire according to arcpy. This script must include also these points.
# 0.1 m is chosen because it's an easy number - the real differences won't be that big.
RBX_distances = {220: {"red": 1.6, "orange": 3, "yellow": 4.1},
                 300: {"red": 1.6, "orange": 3, "yellow": 4.1},
                 400: {"red": 2.7, "orange": 4.3, "yellow": 5.6},
                 500: {"red": 2.7, "orange": 4.3, "yellow": 5.6}}

layerlist = fiona.listlayers(working_gdb)
RBX_gdb_layers = [layer for layer in fiona.listlayers(
    working_gdb) if "RBX_clean" in layer]

# Dictionary for storing <color>_RBX geodataframes from all lines
RBX_all_lines = {"yellow": [], "orange": [], "red": []}
RBX_points_all_lines = {"yellow": [], "orange": [], "red": []}
area_summary = []


powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)

powerlines_df.apply(RBX_polys_stats, axis=1)

# Compute summary statistics per LG (area in square meters of yellow, orange and red RBX)
header = ['LG', 'ledning', 'littera', 'gul', 'orange',
          'röd', 'antal gula', 'antal orange', 'antal röda']
df_area_summary = pd.DataFrame(area_summary, columns=header)

# %%


Doing LG022_1
Clustered_buffers:                                             geometry
0  POLYGON ((609487.122 7018323.139, 609487.101 7...
Clustered_buffers:                                             geometry
0  POLYGON ((609045.417 7016873.003, 609045.384 7...
1  POLYGON ((609046.074 7016878.547, 609046.037 7...
2  POLYGON ((609362.234 7018035.937, 609362.197 7...
Nr of points: 243
Nr of polygons: 12
   polygon_ID AVSTAND_FAS    AVST_H    dz    AVST_F  index_right
0           0              0.436731  4.43  5.444561            0
1           1              0.668028  4.93  5.299389            1
2           2              1.477024  5.75  4.514675            2
3           3              1.779039  4.59  4.721544            3
4           4              1.709682  4.81  4.376525            4
Number of closest points: 12
Original polygon columns: Index(['geometry', 'LG', 'littera', 'polygon_ID'], dtype='object')
Merged polygon columns: Index(['geometry', 'LG', 'littera', 'polygon_ID', 'AVSTAN

[11]:370: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
[11]:370: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
[11]:370: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry c

Doing LG022_2
Clustered_buffers:                                             geometry
0  POLYGON ((650681.698 7089559.581, 650681.69 70...
1  POLYGON ((650670.19 7089559.703, 650670.179 70...
2  POLYGON ((650693.23 7089586.513, 650693.219 70...
Nr of points: 3
Nr of polygons: 3
   polygon_ID AVSTAND_FAS    AVST_H    dz    AVST_F  index_right
0           0              0.889036  4.25  5.388875            0
1           1              0.443775  4.89  5.376464            1
2           2              0.630216  3.51  5.428878            2
Number of closest points: 3
Original polygon columns: Index(['geometry', 'LG', 'littera', 'polygon_ID'], dtype='object')
Merged polygon columns: Index(['geometry', 'LG', 'littera', 'polygon_ID', 'AVSTAND_FAS', 'AVST_H',
       'dz', 'AVST_F', 'index_right'],
      dtype='object')
Columns: Index(['LG', 'littera', 'AVST_F', 'AVST_H', 'geometry', 'dz'], dtype='object')


[11]:370: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
[11]:370: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
[11]:370: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry c

# Put RBX shape polygons in gdb

Skriver över geometrier från shapefiler till geodatabaser. <br>
Polygoner, all_points och closest_points hämtas från shapefiler och skrivs till geodatabasen, <br>
feature datasets
    * RBX_polygons
    * RBX_closest_points
    * RBX_all_points
Den tar alla filer i mappen, så om även de gamla ligger i samma mapp kommer dessa att köras igen, vilket tar onödig tid. Dessutom måste de då plockas bort efteråt.


In [12]:
import arcpy
import os

# Paths to feature classes
RBX_polygons_fd = os.path.join(working_gdb,"RBX_polygons")                                                                                            # kolla även det ör som gäller vid utleverans om det är så att databasfilen ska kopieras över till utleverans
RBX_closest_points_fd = os.path.join(working_gdb,"RBX_closest_points")
RBX_all_points_fd = os.path.join(working_gdb,"RBX_all_points")

# Path to shapefile folder
#shp_folder = os.path.join(prj_dir, "RBX_polygons")


# all_points
shp_files = [os.path.join(RBX_shape_folder, f) for f in os.listdir(RBX_shape_folder) if f.endswith("_all_points.shp")]

# Loopa över listan och kopiera varje shapefil till geodatabasen
for shp_path in shp_files:
    # Skapa ett namn för featuren i geodatabasen (utan filändelse)
    feature_name = os.path.splitext(os.path.basename(shp_path))[0]
    
    # Kopiera shapefilen till geodatabasen
    arcpy.conversion.FeatureClassToFeatureClass(shp_path, RBX_all_points_fd, feature_name)
    print(f"Kopierade {shp_path} till {RBX_all_points_fd} som {feature_name}")


# closest_points
shp_files = [os.path.join(RBX_shape_folder, f) for f in os.listdir(RBX_shape_folder) if f.endswith("_closest_points.shp")]

# Loopa över listan och kopiera varje shapefil till geodatabasen
for shp_path in shp_files:
    # Skapa ett namn för featuren i geodatabasen (utan filändelse)
    feature_name = os.path.splitext(os.path.basename(shp_path))[0]
    
    # Kopiera shapefilen till geodatabasen
    arcpy.conversion.FeatureClassToFeatureClass(shp_path, RBX_closest_points_fd, feature_name)
    print(f"Kopierade {shp_path} till {RBX_closest_points_fd} som {feature_name}")
    

# polygons
shp_files = [os.path.join(RBX_shape_folder, f) for f in os.listdir(RBX_shape_folder) if f.endswith("_RBX.shp")]

# Loopa över listan och kopiera varje shapefil till geodatabasen
for shp_path in shp_files:
    # Skapa ett namn för featuren i geodatabasen (utan filändelse)
    feature_name = os.path.splitext(os.path.basename(shp_path))[0]
    
    # Kopiera shapefilen till geodatabasen
    arcpy.conversion.FeatureClassToFeatureClass(shp_path, RBX_polygons_fd, feature_name)
    print(f"Kopierade {shp_path} till {RBX_polygons_fd} som {feature_name}")

Kopierade C:/SVK_2024/pythonkörningar\RBX_shp_250128\LG022_1_all_points.shp till C:/SVK_2024/pythonkörningar\working_250128.gdb\RBX_all_points som LG022_1_all_points
Kopierade C:/SVK_2024/pythonkörningar\RBX_shp_250128\LG022_2_all_points.shp till C:/SVK_2024/pythonkörningar\working_250128.gdb\RBX_all_points som LG022_2_all_points
Kopierade C:/SVK_2024/pythonkörningar\RBX_shp_250128\LG022_1_closest_points.shp till C:/SVK_2024/pythonkörningar\working_250128.gdb\RBX_closest_points som LG022_1_closest_points
Kopierade C:/SVK_2024/pythonkörningar\RBX_shp_250128\LG022_2_closest_points.shp till C:/SVK_2024/pythonkörningar\working_250128.gdb\RBX_closest_points som LG022_2_closest_points
Kopierade C:/SVK_2024/pythonkörningar\RBX_shp_250128\LG022_1_RBX.shp till C:/SVK_2024/pythonkörningar\working_250128.gdb\RBX_polygons som LG022_1_RBX
Kopierade C:/SVK_2024/pythonkörningar\RBX_shp_250128\LG022_2_RBX.shp till C:/SVK_2024/pythonkörningar\working_250128.gdb\RBX_polygons som LG022_2_RBX


# Tar jättelång tid, borde gå att förbättra? Få fälten rätt direkt i nåt tidigare steg? Eller gör när alla featureklasser slagits ihop, om littera är fixat innan

In [13]:
import arcpy, os, glob, shutil, datetime, time
from pathlib import Path
import pandas as pd
from datetime import date
import time

# Paths to feature datasets
RBX_polygons_fd = os.path.join(working_gdb,"RBX_polygons")       # kolla även det ör som gäller vid utleverans om det är så att databasfilen ska kopieras över till utleverans
RBX_closest_points_fd = os.path.join(working_gdb,"RBX_closest_points")
RBX_all_points_fd = os.path.join(working_gdb,"RBX_all_points")

urspr = "SWECO"
ins_met = 20
matosak_plan = 1000
matosak_hojd = 1000
lev_dat = str(date.today())


def fix_fields(powerline):
    LG = powerline["LG"]
    line = powerline["line"]
    littera = powerline["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    LG_date = cvd_DATE[f"{LG}_{line}"]
    print(f"Fixar fält för {LG}_{line}, skannad {LG_date}")

        
    arcpy.env.overwriteOutput = True
    RBX_polygons_fc = f"{LG}_{line}_RBX"
    RBX_closest_points_fc = f"{LG}_{line}_closest_points"
    RBX_all_points_fc = f"{LG}_{line}_all_points"
    
    print(os.path.join(RBX_polygons_fd, RBX_polygons_fc))
    if arcpy.Exists(os.path.join(RBX_polygons_fd, RBX_polygons_fc)):
        
        start_time = time.time()
        # Polygoner
        arcpy.env.workspace = RBX_polygons_fd
        arcpy.management.AddField(RBX_polygons_fc, "LITTERA", "TEXT", field_length=255, field_alias="Littera")
        arcpy.management.CalculateField(RBX_polygons_fc, "LITTERA", "".join(("'",littera,"'")))
        arcpy.management.AddField(RBX_polygons_fc, "LEDNINGSGATA", "LONG", field_alias="Ledningsgata")
        arcpy.management.CalculateField(RBX_polygons_fc, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(RBX_polygons_fc, "RGDTM", "DATE", field_alias="Registreringsdatum")
        arcpy.management.CalculateField(RBX_polygons_fc, "RGDTM", "".join(("'",LG_date,"'")))
        arcpy.management.AddField(RBX_polygons_fc, "Ursprung", "TEXT")
        arcpy.management.CalculateField(RBX_polygons_fc, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(RBX_polygons_fc, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(RBX_polygons_fc, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(RBX_polygons_fc, "Insamlingsmetod", "LONG", field_alias="Inventeringsmetod")
        arcpy.management.CalculateField(RBX_polygons_fc, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(RBX_polygons_fc, "Matosakerhet_Plan", "LONG", field_alias="Mätosäkerhet i plan")
        arcpy.management.CalculateField(RBX_polygons_fc, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(RBX_polygons_fc, "Matosakerhet_Hojd", "LONG", field_alias="Mätosäkerhet i höjd")
        arcpy.management.CalculateField(RBX_polygons_fc, "Matosakerhet_Hojd", int(matosak_hojd))
        arcpy.management.AddField(RBX_polygons_fc, "KLASS", "SHORT", field_alias="Klass")        
        arcpy.management.CalculateField(RBX_polygons_fc, "KLASS", "!KLASS_TEMP!", "PYTHON3")
        arcpy.management.DeleteField(RBX_polygons_fc, "KLASS_TEMP")
        arcpy.management.AlterField(RBX_polygons_fc, "AVST_F", "AVSTAND_FAS", new_field_alias="Avstånd till fas [m]")
        arcpy.management.AlterField(RBX_polygons_fc, "AVST_H", "AVSTAND_HORISONTELLT", new_field_alias="Horisontellt avstånd till fas [m]")
        arcpy.management.AlterField(RBX_polygons_fc, "dz", "DELTA_HOJD", new_field_alias="Trädhöjd [m]")
        
        arcpy.management.AssignDomainToField(RBX_polygons_fc, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(RBX_polygons_fc, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(RBX_polygons_fc, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_polygons_fc, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_polygons_fc, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
        
        end_time = time.time()
        print("    - polygoner fixade")
        
        elapsed_time = end_time - start_time

        print(f"Skriptet är klart! Det tog {elapsed_time:.2f} sekunder.")
        
        # Closest points
        arcpy.env.workspace = RBX_closest_points_fd

        arcpy.management.AddField(RBX_closest_points_fc, "LITTERA", "TEXT", field_length=255, field_alias="Littera")
        arcpy.management.CalculateField(RBX_closest_points_fc, "LITTERA", "".join(("'",littera,"'")))
        arcpy.management.AddField(RBX_closest_points_fc, "LEDNINGSGATA", "LONG", field_alias="Ledningsgata")
        arcpy.management.CalculateField(RBX_closest_points_fc, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(RBX_closest_points_fc, "RGDTM", "DATE", field_alias="Registreringsdatum")
        arcpy.management.CalculateField(RBX_closest_points_fc, "RGDTM", "".join(("'",LG_date,"'")))
        arcpy.management.AddField(RBX_closest_points_fc, "Ursprung", "TEXT")
        arcpy.management.CalculateField(RBX_closest_points_fc, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(RBX_closest_points_fc, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(RBX_closest_points_fc, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(RBX_closest_points_fc, "Insamlingsmetod", "LONG", field_alias="Inventeringsmetod")
        arcpy.management.CalculateField(RBX_closest_points_fc, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(RBX_closest_points_fc, "Matosakerhet_Plan", "LONG", field_alias="Mätosäkerhet i plan")
        arcpy.management.CalculateField(RBX_closest_points_fc, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(RBX_closest_points_fc, "Matosakerhet_Hojd", "LONG", field_alias="Mätosäkerhet i höjd")
        arcpy.management.CalculateField(RBX_closest_points_fc, "Matosakerhet_Hojd", int(matosak_hojd))
        arcpy.management.AddField(RBX_closest_points_fc, "KLASS", "SHORT", field_alias="Klass")        
        arcpy.management.CalculateField(RBX_closest_points_fc, "KLASS", "!KLASS_TEMP!", "PYTHON3")
        arcpy.management.DeleteField(RBX_closest_points_fc, "KLASS_TEMP")
        arcpy.management.AlterField(RBX_closest_points_fc, "AVST_F", "AVSTAND_FAS", new_field_alias="Avstånd till fas [m]")
        arcpy.management.AlterField(RBX_closest_points_fc, "AVST_H", "AVSTAND_HORISONTELLT", new_field_alias="Horisontellt avstånd till fas [m]")
        arcpy.management.AlterField(RBX_closest_points_fc, "dz", "DELTA_HOJD", new_field_alias="Trädhöjd [m]")
        
        arcpy.management.AssignDomainToField(RBX_closest_points_fc, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(RBX_closest_points_fc, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(RBX_closest_points_fc, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_closest_points_fc, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_closest_points_fc, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
        
        print("    - closest_points fixade")

        
        # All points
        arcpy.env.workspace = RBX_all_points_fd

        arcpy.management.AddField(RBX_all_points_fc, "LITTERA", "TEXT", field_length=255, field_alias="Littera")
        arcpy.management.CalculateField(RBX_all_points_fc, "LITTERA", "".join(("'",littera,"'")))
        arcpy.management.AddField(RBX_all_points_fc, "LEDNINGSGATA", "LONG", field_alias="Ledningsgata")
        arcpy.management.CalculateField(RBX_all_points_fc, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(RBX_all_points_fc, "RGDTM", "DATE", field_alias="Registreringsdatum")
        arcpy.management.CalculateField(RBX_all_points_fc, "RGDTM", "".join(("'",LG_date,"'")))
        arcpy.management.AddField(RBX_all_points_fc, "Ursprung", "TEXT")
        arcpy.management.CalculateField(RBX_all_points_fc, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(RBX_all_points_fc, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(RBX_all_points_fc, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(RBX_all_points_fc, "Insamlingsmetod", "LONG", field_alias="Inventeringsmetod")
        arcpy.management.CalculateField(RBX_all_points_fc, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(RBX_all_points_fc, "Matosakerhet_Plan", "LONG", field_alias="Mätosäkerhet i plan")
        arcpy.management.CalculateField(RBX_all_points_fc, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(RBX_all_points_fc, "Matosakerhet_Hojd", "LONG", field_alias="Mätosäkerhet i höjd")
        arcpy.management.CalculateField(RBX_all_points_fc, "Matosakerhet_Hojd", int(matosak_hojd))
        arcpy.management.AddField(RBX_all_points_fc, "KLASS", "SHORT", field_alias="Klass")        
        arcpy.management.CalculateField(RBX_all_points_fc, "KLASS", "!KLASS_TEMP!", "PYTHON3")
        arcpy.management.DeleteField(RBX_all_points_fc, "KLASS_TEMP")
        arcpy.management.AlterField(RBX_all_points_fc, "AVST_F", "AVSTAND_FAS", new_field_alias="Avstånd till fas [m]")
        arcpy.management.AlterField(RBX_all_points_fc, "AVST_H", "AVSTAND_HORISONTELLT", new_field_alias="Horisontellt avstånd till fas [m]")
        arcpy.management.AlterField(RBX_all_points_fc, "dz", "DELTA_HOJD", new_field_alias="Trädhöjd [m]")
        
        arcpy.management.AssignDomainToField(RBX_all_points_fc, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(RBX_all_points_fc, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(RBX_all_points_fc, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_all_points_fc, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_all_points_fc, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
        
        print("    - all_points fixade")
    
powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
df_cvd_DATE = pd.read_csv(scandate_file, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.Description[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(len(df_cvd_LEDNINGSGATA))}
cvd_DATE = {f"{row['LG']}_{int(row['line'])}": row['skanningsdatum'] for _, row in df_cvd_DATE.iterrows()}

powerlines_df.apply(fix_fields, axis=1)

Fixar fält för LG022_1, skannad 2024-08-06
C:/SVK_2024/pythonkörningar\working_250128.gdb\RBX_polygons\LG022_1_RBX
    - polygoner fixade
Skriptet är klart! Det tog 286.16 sekunder.
    - closest_points fixade
    - all_points fixade
Fixar fält för LG022_2, skannad 2024-08-06
C:/SVK_2024/pythonkörningar\working_250128.gdb\RBX_polygons\LG022_2_RBX
    - polygoner fixade
Skriptet är klart! Det tog 281.62 sekunder.
    - closest_points fixade
    - all_points fixade


0    None
1    None
dtype: object

In [ ]:
df_cvd_DATE = pd.read_csv(scandate_file, sep="\t", header=0)
cvd_DATE = {f"{row['LG']}_{int(row['line'])}": row['skanningsdatum'] for _, row in df_cvd_DATE.iterrows()}
print(df_cvd_DATE)

# Komplettering lagernamn
Eftersom shapefiler inte kan fältnamn längre än 10 symboler, kompletter detta skript lagernamnen enligt<br>
SVKs manual.<br>
Ändrar även namn på dz till DELTA_HOJD, vilken kan bytas ut i det externa skriptet när tid ges.

In [9]:
#import arcpy, os, glob, SVK, shutil
#from pathlib import Path
#import pandas as pd
#
#RBX_shp_dir = os.path.join(prj_dir,"RBX_polygons")
#RBX_polygons_fd = os.path.join(gdb,"RBX_polygons")                                                                                            
#RBX_closest_points_fd = os.path.join(gdb,"RBX_closest_points")
#RBX_all_points_fd = os.path.join(gdb,"RBX_all_points")
#
#def shp2gdb(powerline):
#    LG = powerline[0]
#    line = powerline[1]
#    
#    RBX_polygons_shp_path = os.path.join(RBX_shp_dir,f"{LG}_{line}_RBX.shp")
#    RBX_closest_points_shp_path = os.path.join(RBX_shp_dir,f"{LG}_{line}_closest_points.shp")
#    RBX_all_points_shp_path = os.path.join(RBX_shp_dir,f"{LG}_{line}_all_points.shp")
#    RBX_polygons_fc = f"RBX_polygon_{LG}_{line}"
#    RBX_closest_points_fc = f"RBX_closest_points_{LG}_{line}"
#    RBX_all_points_fc = f"RBX_all_points_{LG}_{line}"
#    ch_fd_lst = [RBX_polygons_fd, RBX_closest_points_fd, RBX_all_points_fd]
#    ch_fc_lst = [RBX_polygons_fc, RBX_closest_points_fc,RBX_all_points_fc]
#    
#    if os.path.exists(RBX_polygons_shp_path):
#        for fd, fc in zip(ch_fd_lst, ch_fc_lst):
#            print(f"Altering fields for {LG}_{line}")
#            path_joined = os.path.join(fd, fc)
#            field_names = [f.name for f in arcpy.ListFields(path_joined)]
#            if "AVSTAND_HO" in field_names: 
#                print(f"Altering field for {LG}_{line}")
#                arcpy.management.AlterField(
#                    path_joined,
#                    "AVSTAND_HO",
#                    "AVSTAND_HORISONTELLT",
#                    "AVSTAND_HORISONTELLT")
#            if "AVSTAND_FA" in field_names:    
#                arcpy.management.AlterField(
#                    path_joined,
#                    "AVSTAND_FA",
#                    "AVSTAND_FAS",
#                    "AVSTAND_FAS")
#            if "dz" in field_names:    
#                arcpy.management.AlterField(
#                    path_joined,
#                    "dz",
#                    "DELTA_HOJD",
#                    "DELTA_HOJD")
#    else:
#        print(f"No RBX for {LG}_{line}")
#    
#LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
#LGs_info.apply(shp2gdb, axis=1)

Altering fields for LG026_1
Altering field for LG026_1
Altering fields for LG026_1
Altering field for LG026_1
Altering fields for LG026_1
Altering field for LG026_1
Altering fields for LG026_2
Altering field for LG026_2
Altering fields for LG026_2
Altering field for LG026_2
Altering fields for LG026_2
Altering field for LG026_2
No RBX for LG026_3
Altering fields for LG026_4
Altering field for LG026_4
Altering fields for LG026_4
Altering field for LG026_4
Altering fields for LG026_4
Altering field for LG026_4
Altering fields for LG026_6
Altering field for LG026_6
Altering fields for LG026_6
Altering field for LG026_6
Altering fields for LG026_6
Altering field for LG026_6
No RBX for LG040_1
No RBX for LG040_2
No RBX for LG040_3
No RBX for LG040_4
No RBX for LG040_5
No RBX for LG040_6
No RBX for LG040_7
No RBX for LG040_8
Altering fields for LG041_1
Altering field for LG041_1
Altering fields for LG041_1
Altering field for LG041_1
Altering fields for LG041_1
Altering field for LG041_1
Alte

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
dtype: object

## Slå ihop de olika ledningarnas featureklasser till en

In [14]:
import arcpy

results_gdb = os.path.join(local_dir, f"results_{run_ID}.gdb")

# RBX ALL POINTS
target_fc = os.path.join(results_gdb, "RBX_all_points")
# Sätt workspace till working GDB\RBX_all_points
arcpy.env.workspace = os.path.join(working_gdb, "RBX_all_points")
# lista featureklasser att kopiera
feature_classes = arcpy.ListFeatureClasses()
# Gå igenom varje featureklass och append till target GDB
for fc in feature_classes:
    arcpy.Append_management(inputs=fc, target=target_fc, schema_type="NO_TEST")

# RBX CLOSEST POINTS
target_fc = os.path.join(results_gdb, "RBX_closest_points")
# Sätt workspace till working GDB\RBX_closest_points
arcpy.env.workspace = os.path.join(working_gdb, "RBX_closest_points")
# lista featureklasser att kopiera
feature_classes = arcpy.ListFeatureClasses()
# Gå igenom varje featureklass och append till target GDB
for fc in feature_classes:
    arcpy.Append_management(inputs=fc, target=target_fc, schema_type="NO_TEST")
    
# RBX POLYGONS
target_fc = os.path.join(results_gdb, "RBX_polygons")
# Sätt workspace till working GDB\RBX_polygons
arcpy.env.workspace = os.path.join(working_gdb, "RBX_polygons")
# lista featureklasser att kopiera
feature_classes = arcpy.ListFeatureClasses()
# Gå igenom varje featureklass och append till target GDB
for fc in feature_classes:
    arcpy.Append_management(inputs=fc, target=target_fc, schema_type="NO_TEST")  